# Filtros para limpiar el dataset

In [1]:
import pandas as pd

In [2]:
df_curated = pd.read_csv('../data/curated/curated_database.csv', encoding='utf-8-sig', index_col=[0])

### Que la empresa se encuentre en el contenido

In [48]:
len(df_curated)

2329

In [56]:
empresas = df_curated.loc[:, 'Empresa'].unique()
no_cont = []

for emp in empresas:
    df_empresa = df_curated[df_curated['Empresa'] == emp]
    df_empresa['Contenido'] = df_empresa['Contenido'].str.lower()
    if emp == 'Grupo Exito':
        emp = 'Grupo Éxito'
        no_cont += df_empresa[~df_empresa['Contenido'].str.contains(emp.lower())].index.to_list()
    else:
        no_cont += df_empresa[~df_empresa['Contenido'].str.contains(emp.lower())].index.to_list()
    # no_cont += df_empresa[~df_empresa['Contenido procesado'].str.contains(emp.lower())].index.to_list()

C:\Users\andres.ospina\AppData\Local\Temp\ipykernel_27644\1120787464.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empresa['Contenido'] = df_empresa['Contenido'].str.lower()


In [58]:
#Se debe hacer antes de palabras y tópicos porque cambian los indices
df_curated = df_curated.drop(no_cont).reset_index(drop=True)

In [ ]:
df_curated.to_csv('../data/curated/curated_database.csv', encoding='utf-8-sig')

### Eliminar las palabras que aparecen en mas del n% de los artículos

In [8]:
df_palabras = pd.read_csv('../data/curated/palabras.csv', encoding='utf-8-sig', index_col=[0])

In [15]:
aux = df_palabras.groupby('Palabra').count()
eliminar = aux[aux['Frecuencia'] > round(len(df_curated)*0.2)].index

In [20]:
aux.sort_values(by='Frecuencia', ascending=False).head(40).tail(20)

,Frecuencia,ID_Articulo,Entidad
Palabra,,,
leer,167,167,167
horas,167,167,167
situación,167,167,167
nivel,166,166,166
sociales,164,164,164
seguridad,163,163,163
usuarios,162,162,162
desarrollo,161,161,161
colombiano,160,160,160


In [16]:
eliminar

Index([], dtype='object', name='Palabra')

In [6]:
df_palabras = df_palabras[~df_palabras['Palabra'].isin(eliminar)]

In [28]:
len(df_palabras)

187263

In [7]:
df_palabras.to_csv('../data/curated/palabras.csv', encoding='utf-8-sig')

In [89]:
def elminar_mas_frecuentes(df: pd.DataFrame, agrupar_por: str, columna: str, n: int, prop: float):
    conteo = df.groupby(agrupar_por).count()
    eliminar = conteo[conteo[columna] > round(n*prop)].index
    df = [~df[agrupar_por].isin(eliminar)]
    return df